In [9]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import polars as pl

from wtg_power_prediction.dataset import load_training_dataset, load_turbine_metadata

pio.templates.default = "plotly_dark"

In [10]:
def plot_turbine_map(*, target: int = 1) -> go.Figure:
    turbines_in_competition = [1, 2, 3, 4, 5, 7]
    turbines_in_competition.remove(target)

    df_turbine_metadata = (
        load_turbine_metadata()
        .with_columns(
            pl.when(pl.col("Turbine Name").eq(f"T{target:02}"))
            .then(pl.lit("Target"))
            .when(pl.col("Turbine Name").is_in([f"T{t:02}" for t in turbines_in_competition]))
            .then(pl.lit("Reference"))
            .otherwise(pl.lit("Other"))
            .alias("Turbine Type")
        )
        .collect()
    )

    turbine_map = px.scatter_map(
        df_turbine_metadata,
        lat="Latitude",
        lon="Longitude",
        hover_data={
            "Latitude": False,
            "Longitude": False,
            "Turbine Type": False,
            "Turbine Name": True,
            "Hub Height (m)": True,
            "Rotor Diameter (m)": True,
        },
        color="Turbine Type",
        zoom=12,
    )
    turbine_map.update_traces(marker={"size": 10})
    turbine_map.update_layout(
        width=600,
        legend_title_text=None,
        legend={
            "orientation": "h",
            "yanchor": "bottom",
            "y": 1.02,
            "xanchor": "right",
            "x": 1,
        },
    )
    return turbine_map

In [11]:
def plot_wind_rose(*, height: int = 59) -> go.Figure:
    interpolation_parameter = (height - 10) / (100 - 10)

    df = (
        load_training_dataset()
        .select("ERA5_wind_speed_10m", "ERA5_wind_speed_100m", "ERA5_wind_direction_10m", "ERA5_wind_direction_100m")
        .with_columns(
            pl.col("ERA5_wind_speed_10m")
            .add(pl.col("ERA5_wind_speed_100m").sub(pl.col("ERA5_wind_speed_10m")).mul(interpolation_parameter))
            .alias(f"ERA5_wind_speed_{height}m"),
            pl.col("ERA5_wind_direction_10m")
            .add(
                pl.col("ERA5_wind_direction_100m")
                .sub(pl.col("ERA5_wind_direction_10m"))
                .add(180)
                .mod(360)
                .sub(180)
                .mul(interpolation_parameter)
            )
            .mod(360)
            .alias(f"ERA5_wind_direction_{height}m"),
        )
        .with_columns(
            pl.col(f"ERA5_wind_direction_{height}m")
            .cut(
                breaks=pl.linear_space(0, 337.5, 16, eager=True) + 11.25,
                labels=[
                    "N",
                    "NNE",
                    "NE",
                    "ENE",
                    "E",
                    "ESE",
                    "SE",
                    "SSE",
                    "S",
                    "SSW",
                    "SW",
                    "WSW",
                    "W",
                    "WNW",
                    "NW",
                    "NNW",
                    "N",
                ],
            )
            .alias("Wind Direction"),
            pl.col(f"ERA5_wind_speed_{height}m")
            .cut(
                breaks=[2, 4, 6, 8, 10, 12, 14, 16],
                labels=["0-2", "2-4", "4-6", "6-8", "8-10", "10-12", "12-14", "14-16", "16+"],
            )
            .alias("Wind Speed (m/s)"),
        )
        .collect()
    )

    total_count = len(df)

    df = (
        df.lazy()
        .group_by(["Wind Direction", "Wind Speed (m/s)"])
        .agg(pl.len().alias("Count"))
        .with_columns(
            pl.col("Count").truediv(total_count).mul(100).alias("Frequency"),
        )
        .sort("Wind Direction", "Wind Speed (m/s)")
        .collect()
    )

    return px.bar_polar(
        df,
        r="Frequency",
        theta="Wind Direction",
        color="Wind Speed (m/s)",
        color_discrete_sequence=px.colors.sequential.Plasma_r,
        width=600,
    )

In [12]:
plot_turbine_map(target=1).show()
plot_wind_rose(height=59).show()